# 20240922

1. OCM과 동일하게 WK, RK처리,  M3S계산
    OCM 처리방식 : X_REG + MRC_X 상태로 계산해줌. (당연히 K MRC는 DECORRECT해줌. ) 
    MRC_X 에 PSM, POINT_MRC가 다 들어가있음. ( MRC_X = -PSM + Point MRC ) 
    OCM과 동일하게 한다면 ?  ADI에서는 X_REG_demrc + MRC_X 로 계산해주면 됨.  ※ OCO에서는 PSM INPUT 빼주면 안됨. 순수 X_REG로만 M3S 계산하면됨.

    



In [1]:
import pandas as pd
import numpy as np
import os
from openpyxl import load_workbook
import openpyxl

# nau 파일이 있는 폴더 경로
folder_path = 'C:/py_data/nau'

# 추출할 컬럼 위치 설정 (예: 첫 번째 열은 0, 두 번째 열은 1로 인덱스 시작)
columns_to_extract = [0, 1, 2, 3, 4, 5, 6, 7]  # Wafer, TEST, DieX, DieY, X_reg, Y_reg, MRC_X, MRC_Y의 열 위치

def process_nau_files(folder_path, columns_to_extract):
    # 결과를 담을 리스트 생성
    combined_rawdata_list = []
    combined_trocs_input_list = []
    mrc_data_list = []

    # 폴더 내 모든 nau 파일에 대해 반복
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.nau'):
            file_path = os.path.join(folder_path, file_name)
            
            # 필요한 시트만 읽기
            rawdata_file = pd.read_excel(file_path, sheet_name='RawData-1')
            trocs_input_file = pd.read_excel(file_path, sheet_name='Trocs Input')

            ######################################## RawData-1 전처리 ##################################################
            # 지정된 열 추출
            extracted_data_raw = rawdata_file.iloc[:, columns_to_extract].copy()

            # 'LOT_ID' 값 및 'STEPSEQ' 추출
            lot_id_value_raw = rawdata_file.columns[13]
            stepseq_value_raw = rawdata_file.iloc[0, 13]
            wafer_value_raw = rawdata_file.iloc[0, 0]
            p_eqpid__value_raw = rawdata_file.iloc[1, 13]
            photo_ppid_value_raw = rawdata_file.iloc[11, 13]
            p_time_value_raw = rawdata_file.iloc[2, 13]
            m_time_value_raw = rawdata_file.iloc[4, 13]
            chuckid_value_raw = rawdata_file.iloc[15, 13]

            

                
            # 새로운 컬럼 추가
            extracted_data_raw['STEPSEQ'] = stepseq_value_raw
            extracted_data_raw['LOT_ID'] = lot_id_value_raw

            # 추가 정보 추출 및 컬럼 추가
            extracted_data_raw['STEP_PITCH_X'] = rawdata_file.iloc[6, 13]
            extracted_data_raw['STEP_PITCH_Y'] = rawdata_file.iloc[7, 13]
            extracted_data_raw['MAP_SHIFT_X'] = rawdata_file.iloc[8, 13]
            extracted_data_raw['MAP_SHIFT_Y'] = rawdata_file.iloc[9, 13]

            # 'coordinate_X', 'coordinate_Y' 매핑
            coord_map = rawdata_file[['Test No', 'coordinate_X', 'coordinate_Y']].drop_duplicates(subset='Test No').set_index('Test No')
            extracted_data_raw['coordinate_X'] = extracted_data_raw['TEST'].map(coord_map['coordinate_X'])
            extracted_data_raw['coordinate_Y'] = extracted_data_raw['TEST'].map(coord_map['coordinate_Y'])
                  
            # 'wf_x' 및 'wf_y' 계산
            extracted_data_raw['wf_x'] = (
                extracted_data_raw['DieX'] * extracted_data_raw['STEP_PITCH_X'] +
                extracted_data_raw['MAP_SHIFT_X'] + extracted_data_raw['coordinate_X']
            )
            extracted_data_raw['wf_y'] = (
                extracted_data_raw['DieY'] * extracted_data_raw['STEP_PITCH_Y'] +
                extracted_data_raw['MAP_SHIFT_Y'] + extracted_data_raw['coordinate_Y']
            )

            ##### context 정보추가 #####
            extracted_data_raw['P_EQPID'] = rawdata_file.iloc[1, 13]
            extracted_data_raw['P_TIME'] = rawdata_file.iloc[2, 13]
            extracted_data_raw['M_TIME'] = rawdata_file.iloc[4, 13]
            extracted_data_raw['Photo_PPID'] = rawdata_file.iloc[11, 13]
            extracted_data_raw['Base_EQP1'] = rawdata_file.iloc[12, 13]
            extracted_data_raw['ChuckID'] = rawdata_file.iloc[15, 13]
            extracted_data_raw['ReticleID'] = rawdata_file.iloc[16, 13]
            
            

            # 'Unique_ID'라는 새로운 컬럼 추가
            extracted_data_raw['UNIQUE_ID'] = extracted_data_raw.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)


            # 'Unique_ID2 추가 (TEST, DieX, DieY 추가. 정렬용)
            extracted_data_raw['UNIQUE_ID2'] = extracted_data_raw.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['LOT_ID']}_{row['Wafer']}_{row['TEST']}_{row['DieX']}_{row['DieY']} ", axis=1)
            
                       



            # 컬럼 순서 재조정
            cols_order = [
                'UNIQUE_ID', 'UNIQUE_ID2',
                'STEPSEQ', 'LOT_ID', 'Wafer', 
                'P_EQPID', 'Photo_PPID', 'P_TIME', 'M_TIME', 'ChuckID', 'ReticleID', 'Base_EQP1', 
                'TEST', 'DieX', 'DieY',
                'X_reg', 'Y_reg', 'MRC_X', 'MRC_Y', 
                'STEP_PITCH_X', 'STEP_PITCH_Y', 'MAP_SHIFT_X', 'MAP_SHIFT_Y', 'coordinate_X', 'coordinate_Y', 'wf_x', 'wf_y'
            ]
            extracted_data_raw = extracted_data_raw[cols_order]

            # 리스트에 추가
            combined_rawdata_list.append(extracted_data_raw)







            ######################################## Trocs Input 전처리 ##################################################

            trocs_input_file['STEPSEQ'] = stepseq_value_raw
            trocs_input_file['LOT_ID'] = lot_id_value_raw
            trocs_input_file['Wafer'] = wafer_value_raw
            trocs_input_file['P_EQPID'] = p_eqpid__value_raw
            trocs_input_file['Photo_PPID'] = photo_ppid_value_raw
            trocs_input_file['P_TIME'] = p_time_value_raw 
            trocs_input_file['M_TIME'] = m_time_value_raw 
            trocs_input_file['ChuckID'] = chuckid_value_raw

         
            # Trocs Input 데이터프레임에 'Unique_ID'라는 새로운 컬럼 추가
            trocs_input_file['UNIQUE_ID'] = trocs_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)

            # 'Unique_ID2 추가 (DieX(=dCol), DieY(=dRow) 추가. 정렬용)
            trocs_input_file['UNIQUE_ID2'] = trocs_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['LOT_ID']}_{row['Wafer']}_{row['dCol']}_{row['dRow']}", axis=1)



            
            # 순서 재조정
            # 기존 컬럼 뒤로 밀기 - 순서를 맨 앞으로 추가하는 순서로 재배열
            cols_to_insert = ['UNIQUE_ID', 'UNIQUE_ID2', 'STEPSEQ', 'LOT_ID', 'Wafer', 'P_EQPID', 'Photo_PPID', 'P_TIME', 'M_TIME', 'ChuckID']

            # 각 컬럼을 지정된 위치에 삽입 (맨 앞에)
            for i, col in enumerate(cols_to_insert):
                trocs_input_file.insert(i, col, trocs_input_file.pop(col))

            # 리스트에 추가
            combined_trocs_input_list.append(trocs_input_file)





            ######################################## MRC 전처리 ##################################################

            
            # MRC 데이터 추출을 위해 'RawData-1' 시트를 header=None으로 다시 읽음
            rawdata_file_no_header = pd.read_excel(file_path, sheet_name='RawData-1', header=None)

            # MRC 데이터 추출
            mrc_part1 = rawdata_file_no_header.iloc[0:20, 15:17]
            mrc_part2 = rawdata_file_no_header.iloc[22:40, 15:17]
            mrc_part = pd.concat([mrc_part1, mrc_part2], ignore_index=True)

            # 컬럼 이름 설정
            mrc_part.columns = ['K PARA', 'GPM']

            # INDEX 컬럼 추가 (1부터 시작하는 순차적 번호)
            mrc_part['INDEX'] = range(1, len(mrc_part) + 1)


            # 컬럼 추가             
            mrc_part['STEPSEQ'] = stepseq_value_raw
            mrc_part['LOT_ID'] = lot_id_value_raw
            mrc_part['Wafer'] = wafer_value_raw
            mrc_part['P_EQPID'] = p_eqpid__value_raw
            mrc_part['Photo_PPID'] = photo_ppid_value_raw
            mrc_part['P_TIME'] = p_time_value_raw 
            mrc_part['M_TIME'] = m_time_value_raw 
            mrc_part['ChuckID'] = chuckid_value_raw

            # mrc_part 데이터프레임에 'Unique_ID'라는 새로운 컬럼 추가
            mrc_part['UNIQUE_ID'] = mrc_part.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)


            # 컬럼 순서 재조정
            mrc_cols_order = [
                'UNIQUE_ID',
                'STEPSEQ', 'LOT_ID', 'Wafer', 
                'P_EQPID', 'Photo_PPID', 'P_TIME', 'M_TIME', 'ChuckID',  
                'K PARA', 'GPM', 'INDEX'
            ]
            mrc_part = mrc_part[mrc_cols_order]

            
            # 리스트에 추가
            mrc_data_list.append(mrc_part)




    # 리스트를 데이터프레임으로 병합
    combined_rawdata = pd.concat(combined_rawdata_list, ignore_index=True)
    combined_trocs_input = pd.concat(combined_trocs_input_list, ignore_index=True)
    mrc_data = pd.concat(mrc_data_list, ignore_index=True)

    # 병합 후, UNIQUE_ID기준으로 오름차순 정렬 (내림차순하고 싶으면 ascending=False 옵션 추가하면 됨.) 
    combined_rawdata = combined_rawdata.sort_values(by=['UNIQUE_ID', 'TEST', 'DieX', 'DieY'])    
    combined_trocs_input = combined_trocs_input.sort_values(by=['UNIQUE_ID', 'dCol', 'dRow'])
    mrc_data = mrc_data.sort_values(by=['UNIQUE_ID', 'INDEX'])

    # 최종 데이터를 엑셀 파일로 저장
    with pd.ExcelWriter('output.xlsx') as writer:
        combined_rawdata.to_excel(writer, sheet_name='RawData-1', index=False)
        combined_trocs_input.to_excel(writer, sheet_name='Trocs Input', index=False)
        mrc_data.to_excel(writer, sheet_name='MRC', index=False)



################################### K MRC fit & Decorrect #####################################################################



def kmrc_decorrect(df_rawdata, df_mrc_input):
    # UNIQUE_ID별로 그룹화
    grouped = df_rawdata.groupby('UNIQUE_ID')

    # 예측 결과를 저장할 리스트
    mrc_list = []

    for unique_id, group in grouped:
        stepseq = group['STEPSEQ']
        lot_id = group['LOT_ID']
        wafer = group['Wafer']
        p_eqpid = group['P_EQPID']
        photo_ppid = group['Photo_PPID']
        p_time = group['P_TIME']
        m_time = group['M_TIME']
        chuckid = group['ChuckID']
        reticleid = group['ReticleID']
        base_eqp1 = group['Base_EQP1']
        test = group['TEST']
        die_x = group['DieX']
        die_y = group['DieY']
        mrc_x = group['MRC_X']
        mrc_y = group['MRC_Y']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        coordinate_x = group['coordinate_X']
        coordinate_y = group['coordinate_Y']   
        wf_x = group['wf_x']
        wf_y = group['wf_y']        
        x_reg = group['X_reg']
        y_reg = group['Y_reg']


        
        x = die_x * step_pitch_x + map_shift_x
        y = die_y * step_pitch_y + map_shift_y
        rx = coordinate_x
        ry = coordinate_y

        X_dx = np.vstack([
            np.ones(len(x)), x/1e6, -y/1e6, (x**2)/1e12, (x*y)/1e12, (y**2)/1e12, (x**3)/1e15, (x**2*y)/1e15, (x*y**2)/1e15, (y**3)/1e15, 
            rx/1e6, -ry/1e6, (rx**2)/1e9, (rx*ry)/1e9, (ry**2)/1e9, (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12, (ry**3)/1e12
        ]).T
        X_dy = np.vstack([
            np.ones(len(y)), y/1e6, x/1e6, (y**2)/1e12, (y*x)/1e12, (x**2)/1e12, (y**3)/1e15, (y**2*x)/1e15, (y*x**2)/1e15, (x**3)/1e15,
            ry/1e6, rx/1e6, (ry**2)/1e9, (ry*rx)/1e9, (rx**2)/1e9, (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12
        ]).T

        # 현재 UNIQUE_ID에 해당하는 MRC 데이터만 선택
        mrc_k_odd = df_mrc_input[
            (df_mrc_input['UNIQUE_ID'] == unique_id) & 
            (df_mrc_input['K PARA'].isin(['W1', 'W3', 'W5', 'W7', 'W9', 'W11', 'W13', 'W15', 'W17', 'W19', 
                                          'R3', 'R5', 'R7', 'R9', 'R11', 'R13', 'R15', 'R17', 'R19']))
        ]

        mrc_k_even = df_mrc_input[
            (df_mrc_input['UNIQUE_ID'] == unique_id) & 
            (df_mrc_input['K PARA'].isin(['W2', 'W4', 'W6', 'W8', 'W10', 'W12', 'W14', 'W16', 'W18', 'W20', 
                                          'R4', 'R6', 'R8', 'R10', 'R12', 'R14', 'R16', 'R18']))
        ]

        # 'K PARA'가 정확히 19개씩 있는지 확인
        if len(mrc_k_odd) != 19 or len(mrc_k_even) != 18:
            print(f"UNIQUE_ID {unique_id}의 MRC 데이터가 부족합니다.")
            continue

        # 'GPM' 값을 numpy 배열로 변환
        mrc_k_odd_values = mrc_k_odd['GPM'].values.astype(float)
        mrc_k_even_values = mrc_k_even['GPM'].values.astype(float)



        # mrc fitting (부호반대처리)
        mrc_fit_x = X_dx.dot(mrc_k_odd_values) * -1
        mrc_fit_y = X_dy.dot(mrc_k_even_values) * -1

        # X_reg 에서 kmrc 빼주기
        X_reg_demrc = group['X_reg'] - mrc_fit_x
        Y_reg_demrc = group['Y_reg'] - mrc_fit_y

        # 결과 저장
        mrc_list.append(pd.DataFrame({

            'UNIQUE_ID' : unique_id,
            'STEPSEQ' : stepseq, 
            'LOT_ID' : lot_id,
            'Wafer' : wafer, 
            'P_EQPID' : p_eqpid,
            'Photo_PPID' : photo_ppid,
            'P_TIME' : p_time,
            'M_TIME' : m_time,
            'ChuckID' : chuckid,
            'ReticleID' : reticleid,
            'Base_EQP1' : base_eqp1,
            'TEST' : test,
            'DieX' : die_x,
            'DieY' : die_y,
            'MRC_X' : mrc_x,
            'MRC_Y' : mrc_y,
            'STEP_PITCH_X' : step_pitch_x,
            'STEP_PITCH_Y' : step_pitch_y,
            'MAP_SHIFT_X' : map_shift_x,
            'MAP_SHIFT_Y' : map_shift_y,
            'coordinate_X' : coordinate_x,
            'coordinate_Y' : coordinate_y,
            'wf_x' : wf_x,
            'wf_y' : wf_y,
            'X_reg' : x_reg,
            'Y_reg' : y_reg, 
            
            'mrc_fit_x': mrc_fit_x,
            'mrc_fit_y': mrc_fit_y,
            'X_reg_demrc': X_reg_demrc,
            'Y_reg_demrc': Y_reg_demrc,
        }))

    # mrc 결과 병합
    df_mrc_de = pd.concat(mrc_list, ignore_index=True)
    #df_mrc_de = pd.concat([df_rawdata.reset_index(drop=True), mrc_list], axis=1)

    return df_mrc_de






###################### PSM Input은 빼주고 Point MRC는 더해주고.   X_reg_demrc + MRC X ###########################


def remove_psm_add_pointmrc(df_rawdata):
    df = pd.DataFrame(df_rawdata)
    df['raw_x'] = df['X_reg_demrc'] + df['MRC_X'] 
    df['raw_y'] = df['Y_reg_demrc'] + df['MRC_Y'] 
    return df





###################### OSR ###########################



def multi_lot_regression(df_rawdata):
    # UNIQUE_ID별로 그룹화
    grouped = df_rawdata.groupby('UNIQUE_ID')

    # 회귀분석 결과를 저장할 리스트
    wkrk_results = []

    # 각 그룹에 대해 처리
    for unique_id, group in grouped:
        die_x = group['DieX']
        die_y = group['DieY']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        field_x = group['coordinate_X']
        field_y = group['coordinate_Y']
        
        # 좌표 계산
        x = die_x * step_pitch_x + map_shift_x
        y = die_y * step_pitch_y + map_shift_y
        rx = field_x
        ry = field_y

        # X_dx, X_dy 행렬 구성
        X_dx = np.vstack([
            np.ones(len(x)), x/1e6, -y/1e6, (x**2)/1e12, (x*y)/1e12, (y**2)/1e12, (x**3)/1e15, (x**2*y)/1e15, (x*y**2)/1e15, (y**3)/1e15, 
            rx/1e6, -ry/1e6, (rx**2)/1e9, (rx*ry)/1e9, (ry**2)/1e9, (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12, (ry**3)/1e12
        ]).T
        X_dy = np.vstack([
            np.ones(len(y)), y/1e6, x/1e6, (y**2)/1e12, (y*x)/1e12, (x**2)/1e12, (y**3)/1e15, (y**2*x)/1e15, (y*x**2)/1e15, (x**3)/1e15,
            ry/1e6, rx/1e6, (ry**2)/1e9, (ry*rx)/1e9, (rx**2)/1e9, (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12
        ]).T

        # 종속변수
        Y_dx = group['raw_x']
        Y_dy = group['raw_y']

        # 최소자승법으로 계수 계산
        coeff_dx = np.linalg.lstsq(X_dx, Y_dx, rcond=None)[0]
        coeff_dy = np.linalg.lstsq(X_dy, Y_dy, rcond=None)[0]

        # 결과 저장
        wkrk_results.append(pd.DataFrame({
            'UNIQUE_ID': [unique_id],
            'WK1': [coeff_dx[0]],
            'WK2': [coeff_dy[0]],
            'WK3': [coeff_dx[1]],
            'WK4': [coeff_dy[1]],
            'WK5': [coeff_dx[2]],
            'WK6': [coeff_dy[2]],
            'WK7': [coeff_dx[3]],
            'WK8': [coeff_dy[3]],
            'WK9': [coeff_dx[4]],
            'WK10': [coeff_dy[4]],
            'WK11': [coeff_dx[5]],
            'WK12': [coeff_dy[5]],
            'WK13': [coeff_dx[6]],
            'WK14': [coeff_dy[6]],
            'WK15': [coeff_dx[7]],
            'WK16': [coeff_dy[7]],
            'WK17': [coeff_dx[8]],
            'WK18': [coeff_dy[8]],
            'WK19': [coeff_dx[9]],
            'WK20': [coeff_dy[9]],
            'RK1': [0],
            'RK2': [0],
            'RK3': [coeff_dx[10]],
            'RK4': [coeff_dy[10]],
            'RK5': [coeff_dx[11]],
            'RK6': [coeff_dy[11]],
            'RK7': [coeff_dx[12]],
            'RK8': [coeff_dy[12]],
            'RK9': [coeff_dx[13]],
            'RK10': [coeff_dy[13]],
            'RK11': [coeff_dx[14]],
            'RK12': [coeff_dy[14]],
            'RK13': [coeff_dx[15]],
            'RK14': [coeff_dy[15]],
            'RK15': [coeff_dx[16]],
            'RK16': [coeff_dy[16]],
            'RK17': [coeff_dx[17]],
            'RK18': [coeff_dy[17]],
            'RK19': [coeff_dx[18]],
            'RK20': [0],
        }))

    # 결과 병합
    combined_results = pd.concat(wkrk_results, ignore_index=True)
    return combined_results






def multi_lot_fitting_residual(df_rawdata, df_coeff):
    # UNIQUE_ID별로 그룹화
    grouped = df_rawdata.groupby('UNIQUE_ID')
    
    # 예측 결과를 저장할 리스트
    predictions_list = []
    
    for unique_id, group in grouped:
        stepseq = group['STEPSEQ']
        lot_id = group['LOT_ID']
        wafer = group['Wafer']
        p_eqpid = group['P_EQPID']
        photo_ppid = group['Photo_PPID']
        p_time = group['P_TIME']
        m_time = group['M_TIME']
        chuckid = group['ChuckID']
        reticleid = group['ReticleID']
        base_eqp1 = group['Base_EQP1']
        test = group['TEST']
        die_x = group['DieX']
        die_y = group['DieY']
        mrc_x = group['MRC_X']
        mrc_y = group['MRC_Y']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        coordinate_x = group['coordinate_X']
        coordiante_y = group['coordinate_Y']   
        wf_x = group['wf_x']
        wf_y = group['wf_y']        



        x_reg = group['X_reg']
        y_reg = group['Y_reg']

        raw_x = group['raw_x']
        raw_y = group['raw_y']
        mrc_fit_x = group['mrc_fit_x']
        mrc_fit_y = group['mrc_fit_y']
        x_reg_demrc = group['X_reg_demrc']
        y_reg_demrc = group['Y_reg_demrc']



      
        
        x = die_x * step_pitch_x + map_shift_x
        y = die_y * step_pitch_y + map_shift_y
        rx = coordinate_x
        ry = coordiante_y

        X_dx = np.vstack([
            np.ones(len(x)), x/1e6, -y/1e6, (x**2)/1e12, (x*y)/1e12, (y**2)/1e12, (x**3)/1e15, (x**2*y)/1e15, (x*y**2)/1e15, (y**3)/1e15, 
            rx/1e6, -ry/1e6, (rx**2)/1e9, (rx*ry)/1e9, (ry**2)/1e9, (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12, (ry**3)/1e12
        ]).T
        X_dy = np.vstack([
            np.ones(len(y)), y/1e6, x/1e6, (y**2)/1e12, (y*x)/1e12, (x**2)/1e12, (y**3)/1e15, (y**2*x)/1e15, (y*x**2)/1e15, (x**3)/1e15,
            ry/1e6, rx/1e6, (ry**2)/1e9, (ry*rx)/1e9, (rx**2)/1e9, (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12
        ]).T

        # 해당 LOT_ID의 계수 추출
        coeffs = df_coeff[df_coeff['UNIQUE_ID'] == unique_id].iloc[0]
        coeff_dx = coeffs[['WK1','WK3','WK5','WK7','WK9','WK11','WK13','WK15','WK17','WK19','RK3','RK5','RK7','RK9','RK11','RK13','RK15','RK17','RK19']].values.astype(float)
        coeff_dy = coeffs[['WK2','WK4','WK6','WK8','WK10','WK12','WK14','WK16','WK18','WK20','RK4','RK6','RK8','RK10','RK12','RK14','RK16','RK18']].values.astype(float)


        # 예측값 계산
        pred_x = X_dx.dot(coeff_dx)
        pred_y = X_dy.dot(coeff_dy)

        # 잔차 계산
        residual_x = group['raw_x'] - pred_x
        residual_y = group['raw_y'] - pred_y

        # 결과 저장
        predictions_list.append(pd.DataFrame({
            'UNIQUE_ID' : unique_id,
            'STEPSEQ' : stepseq, 
            'LOT_ID' : lot_id,
            'Wafer' : wafer, 
            'P_EQPID' : p_eqpid,
            'Photo_PPID' : photo_ppid,
            'P_TIME' : p_time,
            'M_TIME' : m_time,
            'ChuckID' : chuckid,
            'ReticleID' : reticleid,
            'Base_EQP1' : base_eqp1,
            'TEST' : test,
            'DieX' : die_x,
            'DieY' : die_y,
            'MRC_X' : mrc_x,
            'MRC_Y' : mrc_y,
            'STEP_PITCH_X' : step_pitch_x,
            'STEP_PITCH_Y' : step_pitch_y,
            'MAP_SHIFT_X' : map_shift_x,
            'MAP_SHIFT_Y' : map_shift_y,
            'coordinate_X' : coordinate_x,
            'coordinate_Y' : coordiante_y,
            'wf_x' : wf_x,
            'wf_y' : wf_y,

            'X_reg' : x_reg,
            'Y_reg' : y_reg,
            'mrc_fit_x' : mrc_fit_x,
            'mrc_fit_y' : mrc_fit_y,
            'X_reg_demrc' : x_reg_demrc,
            'Y_reg_demrc' : y_reg_demrc,
            'raw_x' : raw_x,
            'raw_y' : raw_y,

            'pred_x': pred_x,
            'pred_y': pred_y,
            'residual_x': residual_x,
            'residual_y': residual_y,
        }))

    # 예측 결과 병합
    df_predictions = pd.concat(predictions_list, ignore_index=True)
    return df_predictions





################################### Residual을 shot별 6para로 Regression 후 fitting값 계산  #####################################################################




def multi_lot_CPE(df_residata):
    # 'shot' 별로 데이터를 그룹화 (고유한 die_x, die_y 조합)
    grouped = df_residata.groupby(['UNIQUE_ID', 'DieX', 'DieY'])
    
    # 회귀분석 결과를 저장할 리스트
    shot_regression_results = []
    
    for (unique_id, die_x, die_y), group in grouped:
        # 독립변수와 종속변수 설정 
        # 독립변수 (shot 좌표)
        rx = group['coordinate_X']
        ry = group['coordinate_Y']  

        # 종속변수 (residual)
        Yx = group['residual_x']
        Yy = group['residual_y']

        # 독립변수 배열 구성 및 최소자승법으로 회귀 계수 구하기
        X_dx = np.vstack([np.ones(len(rx)), rx/1e6, -ry/1e6
                          ]).T
        
        X_dy = np.vstack([np.ones(len(ry)), ry/1e6, rx/1e6
                          ]).T

        coeff_dx = np.linalg.lstsq(X_dx, Yx, rcond=None)[0]
        coeff_dy = np.linalg.lstsq(X_dy, Yy, rcond=None)[0]
    
        # 회귀분석 결과를 리스트에 저장
        shot_regression_results.append({
            'UNIQUE_ID': unique_id,
            'DieX': die_x,
            'DieY': die_y,
            'RK1': coeff_dx[0],
            'RK2': coeff_dy[0],
            'RK3': coeff_dx[1],
            'RK4': coeff_dy[1],
            'RK5': coeff_dx[2],
            'RK6': coeff_dy[2]
        })
    
    # 회귀분석 결과를 새로운 DataFrame으로 변환
    df_cpe = pd.DataFrame(shot_regression_results)
    return df_cpe




def multi_lot_cpe_fitting(df_residata, df_cpe):
    # 결과를 저장할 리스트 생성
    predictions_list = []
        
    # 각 데이터 포인트에 대해 처리
    for idx, row in df_residata.iterrows():
        
        unique_id = row['UNIQUE_ID']
        die_x = row['DieX']
        die_y = row['DieY']
        coordinate_x = row['coordinate_X']
        coordinate_y = row['coordinate_Y']
        residual_x = row['residual_x']
        residual_y = row['residual_y']
        
        rx = coordinate_x
        ry = coordinate_y

        # 해당 LOT_ID, DieX, DieY에 해당하는 회귀계수 찾기
        coeffs = df_cpe[(df_cpe['UNIQUE_ID'] == unique_id) & (df_cpe['DieX'] == die_x) & (df_cpe['DieY'] == die_y)]
        
        if not coeffs.empty:
            coeffs = coeffs.iloc[0]
            # 회귀계수 추출
            RK1 = coeffs['RK1']
            RK2 = coeffs['RK2']
            RK3 = coeffs['RK3']
            RK4 = coeffs['RK4']
            RK5 = coeffs['RK5']
            RK6 = coeffs['RK6']
            
            # 예측값 계산
            cpe_pred_x = RK1 + RK3 * (rx / 1e6) + RK5 * (-ry / 1e6)
            cpe_pred_y = RK2 + RK4 * (ry / 1e6) + RK6 * (rx / 1e6)
            
            # CPE 잔차값 계산
            cpe_resi_x = residual_x - cpe_pred_x 
            cpe_resi_y = residual_y - cpe_pred_y

            # 결과 저장
            predictions_list.append({
                'cpe_pred_x': cpe_pred_x,
                'cpe_pred_y': cpe_pred_y,
                'cpe_resi_x': cpe_resi_x,
                'cpe_resi_y': cpe_resi_y,

            })
        else:
            # 해당하는 회귀계수가 없을 경우 NaN 처리
            predictions_list.append({
                'cpe_pred_x': np.nan,
                'cpe_pred_y': np.nan,
                'cpe_resi_x': np.nan,
                'cpe_resi_y': np.nan
            })
    
    # 결과를 DataFrame으로 변환
    df_predictions = pd.DataFrame(predictions_list)
    # 원본 데이터와 병합
    df_residata = pd.concat([df_residata.reset_index(drop=True), df_predictions], axis=1)
    return df_residata











################################### nau 파일 처리 및 데이터 저장 #####################################################################
process_nau_files(folder_path, columns_to_extract)


################################### MRC Decorrect #####################################################################

# 데이타 불러오기 
df_rawdata = pd.read_excel('output.xlsx', sheet_name='RawData-1')
df_mrc_input = pd.read_excel('output.xlsx', sheet_name='MRC')

# mrc k값을 fitting하고 X_reg에서 빼주기 
df_mrc_de = kmrc_decorrect(df_rawdata, df_mrc_input)

# mrc_de 결과를 엑셀 파일에 저장
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_mrc_de.to_excel(writer, sheet_name='MRC_decorrect', index=False)
 
################################### raw calc #####################################################################

# 데이타 불러오기 
df_rawdata = pd.read_excel('output.xlsx', sheet_name='MRC_decorrect')

# X_reg_demrc + MRC X 
df_raw_calc = remove_psm_add_pointmrc(df_rawdata)

# ideal_trocs 결과를 엑셀 파일에 저장
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_raw_calc.to_excel(writer, sheet_name='raw', index=False)



################################### MULTI LOT REGRESSION #####################################################################

# 데이터 불러오기
df_rawdata = pd.read_excel("output.xlsx", sheet_name='raw')

# 회귀분석 수행
df_coeff = multi_lot_regression(df_rawdata)

# 결과를 엑셀 파일에 저장
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_coeff.to_excel(writer, sheet_name='OSR_K', index=False)



################################### MULTI LOT FITTING, RESIDUAL #####################################################################

# 잔차 계산
df_predictions = multi_lot_fitting_residual(df_rawdata, df_coeff)

# 예측 결과를 엑셀 파일에 저장
with pd.ExcelWriter('output.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_predictions.to_excel(writer, sheet_name='OSR_raw_fit_resi', index=False)

    



############################ MULTI LOT CPE ############################################################################


# 데이터 불러오기
df_residata = pd.read_excel("output.xlsx", sheet_name='OSR_raw_fit_resi')


# CPE 계산
df_cpe = multi_lot_CPE(df_residata)

# 결과를 엑셀 파일에 저장
with pd.ExcelWriter("output.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_cpe.to_excel(writer, sheet_name='CPE', index=False)




############################  MULTI LOT CPE FITTING ###########################################

# multi_lot_cpe_fitting 함수 사용하여 예측값 계산
df_cpe_fit_res = multi_lot_cpe_fitting(df_residata, df_cpe)

# 결과를 엑셀 파일에 저장
with pd.ExcelWriter("output.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_cpe_fit_res.to_excel(writer, sheet_name='CPE_fit_res', index=False)

print("모든 작업이 완료되었습니다.")






        

모든 작업이 완료되었습니다.
